装饰器用于在源码中 “标记” 函数，以某种方式增强函数行为。这是一项强大的功能，但是如果想掌握，必须理解闭包

nonlocal 是新出现的关键字，在 Python 3.0 中引入。作为 Python 程序员，如果严格遵守基于类的面向对象编程方式，即使不知道这个关键字也没事，但是如果想自己实现函数装饰器，那就必须了解闭包的方方面面，因此也就需要知道 nonlocal

这一章中我们主要讨论的话题如下：、

- Python 如何计算装饰器语法
- Python 如何判断变量是不是局部的
- 闭包存在的原因和工作原理
- nonlocal 能解决什么问题

掌握这些知识，可以进一步探讨装饰器:

- 实现行为良好的装饰器
- 标准库中有用的装饰器
- 实现一个参数化装饰器

下面我们先介绍基础知识：

## 基础知识

假如有个 decorate 装饰器

```
@decorate
def target():
    print('running target()')
```

上面的写法与下面效果一样:

```
def target():
    print('running target()')
    
target = decorate(target)
```

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner()

@deco
def target():
    print('running target()')
    
target

running inner()


## Python 何时执行装饰器

装饰器一个关键特性是，它们被装饰的函数定义之后立即运行。这通常是在导入模块（Python 加载模块时），如下面的 register.py 模块

In [ ]:
#!/usr/bin/env python
# encoding: utf-8

registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

In [1]:
# 运行后答案如下
# running register(<function f1 at 0x7fbac67ca6a8>)
# running register(<function f2 at 0x7fbac67ca730>)
# running main()
# registry -> [<function f1 at 0x7fbac67ca6a8>, <function f2 at 0x7fbac67ca730>]
# running f1()
# running f2()
# running f3()

看到 register 在模块中其它函数之前运行（两次）。调用 register 时，传给它的参数是被装饰的函数，例如 <function f1 at 0x7fbac67ca6a8>

加载模块后，registry 有两个被装饰函数的引用：f1 和 f2。这两个函数，以及 f3，只在 main 明确调用它们时候才执行

如果单纯导入 register.py

In [1]:
import register

running register(<function f1 at 0x7f2054630c80>)
running register(<function f2 at 0x7f2054630ea0>)


此时查看 registry 的值：

In [3]:
register.registry

[<function register.f1>, <function register.f2>]

这主要为了强调，函数装饰器在导入模块时立即执行，而被装饰的函数只在明确调用时执行。这突出了 Python 程序员说的 导入时 和 运行时 的区别

考虑到装饰器在真实代码中的常用方式，上面的例子有两个不同寻常的地方：

- 装饰器函数与被装饰的函数在同一个模块定义，实际上装饰器通常在一个模块中定义，然后应用到其它模块中的函数上
- register 装饰器返回的函数与通过参数传入的相同，实际上大多数装饰器会在内部定义一个函数，然后返回

## 使用装饰器改进 “策略” 模式

在上一章的商品折扣例子中有一个问题，每次通过 best_promo 函数判断最大折扣，这个函数也需要一个折扣函数列表，如果忘了添加，会导致一些不容易被发现的问题。下面使用注册装饰器解决了这个问题：

In [4]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    '''为积分为 1000 或以上的顾客提供 5% 折扣'''
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    '''单个商品为 20 个或以上时提供 %10 折扣'''
    
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order_promo(order):
    '''订单中的不同商品达到 10 个或以上时提供 %7 折扣'''
    
def best_promo(order):
    return max(promo(order) for promo in promos)

这个方案有几个优点

- 促销策略函数无需使用特殊名（即不用以 _promo 结尾）
- @promotion 装饰器突出了被装饰函数的作用，还可以临时禁止某个折扣策略，只需要把装饰器注释
- 促销折扣策略可以在其它模块中定义，在系统任意地方都行，只要使用 @promotion 装饰器即可

不过，多数的装饰器会修改被装饰函数。通常，它们会定义一个内部函数，然后将其返回，替换被装饰的函数。使用内部函数的代码几乎都要靠闭包才能正确工作。为了理解闭包，我们先退后一步，了解 Python 中变量的作用域

## 变量作用域规则

In [7]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

这里出现错误不奇怪，如果先给全局变量 b 赋值，然后调用 f1，就不会出错

In [8]:
b = 6
f1(3)

3
6


这也很正常，下面是一个会让你大吃一惊的例子：），前面代码和上面的例子一样，可是，在赋值之前，第二个 print 失败了。

In [9]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

首先输出了 3，表明 print(a) 执行了。但是 print(b) 无法执行，这是因为，Python 编译函数的定义时，判断 b 是局部变量，因为在函数中给它赋值了。生成的字节码证实了这种判断，Python 会尝试从本地环境获取 b。后面调用 f2(3) 时，f2 的定义体会获取并打印局部变量 a 的值，但是尝试获取局部变量 b 的时候，发现 b 没有绑定值。

这不是缺陷，而是设计选择，Python 不要求声明变量，但是假定在函数定义体中赋值的变量是局部变量。

如果在函数中赋值时想让解释器把 b 当成全局变量，要求使用 global 声明：

In [10]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
f3(3)

3
6


In [11]:
f3(3)

3
9


In [12]:
b = 30
b

30

了解了 Python 的变量作用域之后，我们就可以讨论闭包了

## 闭包

人们有时会把闭包和匿名函数弄混，这是有历史原因的，在函数内部定义函数不常见，直到开始使用匿名函数才这么做。而且，只有涉及嵌套函数时才有闭包的问题。因此，很多人是同时知道这两个概念的

其实，闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量。函数是不是匿名的没有关系，关键是它能访问定义体之外定义的非全局变量。

我们通过例子来理解，加入 avg 函数是计算不断增加的系列值的均值，例如整个历史中的某个商品的平均收盘价，每天都会增加新价格，因此平均值要考虑至目前为止的所有价格。

一开始，avg 是这样的：

In [14]:
class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)
    
avg = Averager()
avg(10)

10.0

In [15]:
avg(11)

10.5

In [16]:
avg(12)

11.0

下面是函数式实现，使用高阶函数 make_averager，调用 make_averager，返回一个 averager 函数对象。每次调用 averager 时，都会把参数添加到系列值中，然后计算当前平均值

In [24]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

In [26]:
avg = make_averager()

In [27]:
avg(10)

10.0

In [28]:
avg(11)

10.5

In [29]:
avg(12)

11.0